In [1]:
import datetime
import ee 
import geemap
import colorcet as cc
from data_utils.pygeoboundaries import get_adm_ee

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project='hotspotstoplight')

In [22]:
startDate = '2023-01-01'
endDate = '2023-12-31'

place_name = "Costa Rica"

scale = 90

snake_case_place_name = place_name.replace(' ', '_').lower()

aoi = get_adm_ee(territories=place_name, adm='ADM0')
bbox = aoi.geometry().bounds()

In [23]:
# Applies scaling factors.
def apply_scale_factors(image):
    # Scale and offset values for optical bands
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    
    # Scale and offset values for thermal bands
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    
    # Add scaled bands to the original image
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to Mask Clouds and Cloud Shadows in Landsat 8 Imagery
def cloud_mask(image):
    # Define cloud shadow and cloud bitmasks (Bits 3 and 5)
    cloud_shadow_bitmask = 1 << 3
    cloud_bitmask = 1 << 5
    
    # Select the Quality Assessment (QA) band for pixel quality information
    qa = image.select('QA_PIXEL')
    
    # Create a binary mask to identify clear conditions (both cloud and cloud shadow bits set to 0)
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    
    # Update the original image, masking out cloud and cloud shadow-affected pixels
    return image.updateMask(mask)

In [69]:
def process_year(year):
    # Define the start and end dates for the year
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year, 12, 31)

    # Import and preprocess Landsat 8 imagery for the year
    imageCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                .filterBounds(bbox) \
                .filterDate(startDate, endDate) \
                .map(apply_scale_factors) \
                .map(cloud_mask)

    # Function to calculate LST for each image in the collection
    def calculate_lst(image):
        # Calculate Normalized Difference Vegetation Index (NDVI)
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        
        ndvi = ndvi.set('system:time_start', startDate.millis())
        ndvi_xarray = ndvi.wx.to_xarray()

        ndvi_max = ndvi_xarray['NDVI'].max(skipna=True).values.item()
        ndvi_min = ndvi_xarray['NDVI'].min(skipna=True).values.item()

        # Fraction of Vegetation (FV) Calculation
        fv = ndvi.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).pow(2).rename('FV')

        # Emissivity Calculation
        em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

        # Select Thermal Band (Band 10) and Rename It
        thermal = image.select('ST_B10').rename('thermal')

        # Land Surface Temperature (LST) Calculation
        lst = thermal.expression(
            '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
                'TB': thermal.select('thermal'), # Select the thermal band
                'em': em # Assign emissivity
            }).rename('LST')

        return lst

    # Apply the calculate_lst function to each image in the collection
    lstCollection = imageCollection.map(calculate_lst)

    # Create a binary image for each image in the collection where 1 indicates LST >= 33 and 0 otherwise
    hotDaysCollection = lstCollection.map(lambda image: image.gte(33))

    # Sum all the binary images in the collection to get the total number of hot days in the year
    hotDaysYear = hotDaysCollection.sum()

    landcover = ee.Image("ESA/WorldCover/v100/2020").select('Map').clip(bbox)

    dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic().clip(bbox)

    image_for_sampling = landcover.rename('landcover') \
        .addBands(dem.rename('elevation')) \
        .addBands(ee.Image.pixelLonLat()) \
        .addBands(hotDaysYear) 

    return image_for_sampling

# Define the current year
current_year = datetime.datetime.now().year

# Create a list of years from 2014 to the year before the current year
years = list(range(2014, current_year))

# Initialize an empty list to store valid ee.Image objects
valid_image_collections = []

# Loop over the years from 2014 to the year before the current year
for year in years:
    processed_image = process_year(year)
    
    # Check if the processed_image is an instance of ee.Image
    if isinstance(processed_image, ee.Image):
        valid_image_collections.append(processed_image)
    else:
        print(f"Warning: Year {year} did not return a valid ee.Image object and will be excluded.")

# Convert the list of valid ee.Image objects to an ImageCollection
image_collection = ee.ImageCollection(valid_image_collections)

EEException: A mapped function's arguments cannot be used in client-side operations

In [34]:
import wxee

In [64]:
# Define the start and end dates for the year
startDate = ee.Date.fromYMD(2023, 1, 1)
endDate = ee.Date.fromYMD(2023, 12, 31)

    # Import and preprocess Landsat 8 imagery for the year
image = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
            .filterBounds(bbox) \
            .filterDate(startDate, endDate) \
            .map(apply_scale_factors) \
            .map(cloud_mask) \
            .median() \
            .clip(bbox)

ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')


In [65]:
# set system:time_start property for the image based on the start date
ndvi = ndvi.set('system:time_start', startDate.millis())

In [66]:
ndvi_xarray = ndvi.wx.to_xarray()

Downloading:   0%|          | 0.00/948 [00:00<?, ?iB/s]

In [67]:
# return the max value of the NDVI as a scalar, omitting NaN values
max_ndvi = ndvi_xarray['NDVI'].max(skipna=True).values
min_ndvi = ndvi_xarray['NDVI'].min(skipna=True).values

In [68]:
print(f"NDVI Min: {min_ndvi}, NDVI Max: {max_ndvi}")

NDVI Min: 0.2616705298423767, NDVI Max: 0.8871724605560303


In [52]:
print(min_ndvi)

<xarray.Dataset>
Dimensions:      ()
Coordinates:
    spatial_ref  int32 0
Data variables:
    NDVI         float32 nan


In [30]:
# Define the number of samples per year
num_samples_per_year = 25000 // (current_year - 2014)

samples_feature_collection = ee.FeatureCollection([])

# Loop over each year, process the image, and sample directly
for year in range(2014, current_year):
    try:
        processed_image = process_year(year)
        
        # Directly sample the processed image
        sample = processed_image.sample(
            region=bbox,
            scale=scale,
            numPixels=num_samples_per_year,
            seed=0,
            geometries=True  # Include geometries if needed for visualization
        )
        
        # Aggregate the samples into a FeatureCollection
        samples_feature_collection = samples_feature_collection.merge(sample)
        
    except Exception as e:
        print(f"Skipping year {year} due to an error: {e}")

In [31]:
# Merge the samples to create a single feature collection
training_sample = ee.FeatureCollection(samples_feature_collection)

# Split the data into training and testing
training_sample = training_sample.randomColumn()
training = training_sample.filter(ee.Filter.lt('random', 0.7))
testing = training_sample.filter(ee.Filter.gte('random', 0.7))

# Train the Random Forest regression model
# inputProperties=['NDVI', 'NDBI', 'NDWI', 'EM', 'longitude', 'latitude', 'landcover', 'elevation']
inputProperties=['longitude', 'latitude', 'landcover', 'elevation']
numTrees = 10  # Number of trees in the Random Forest
regressor = ee.Classifier.smileRandomForest(numTrees).setOutputMode('REGRESSION').train(
    training, 
    classProperty='LST', 
    inputProperties=inputProperties
)


In [32]:
recent_image = image_collection.sort('system:time_start', False).first()

predicted_image = recent_image.select(inputProperties).classify(regressor)

difference = recent_image.select('LST').subtract(predicted_image).rename('difference')

In [33]:
# Assuming 'max_lst' is your actual maximum LST image and 'predicted_image' contains the predictions
# Calculate the squared difference between actual and predicted LST
squared_difference = recent_image.select('LST').subtract(predicted_image).pow(2).rename('difference')

# Reduce the squared differences to get the mean squared difference over your area of interest (aoi)
mean_squared_error = squared_difference.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=bbox,
    scale=scale,  # Adjust scale to match your dataset's resolution
    maxPixels=1e14
)

# Calculate the square root of the mean squared error to get the RMSE
rmse = mean_squared_error.getInfo()['difference'] ** 0.5

print('RMSE:', rmse)

EEException: Too many concurrent aggregations.

In [ ]:
vizParams = {
    'min': 0,
    'max': 45,
    'palette': cc.fire
}

m = geemap.Map()
m.centerObject(aoi, 8)
m.add("basemap_selector")
m.add("layer_manager")
m.addLayer(recent_image.select('LST').clip(aoi), vizParams, 'Actual Max LST')
m.addLayer(predicted_image.clip(aoi), vizParams, 'Predicted LST')
m.addLayer(difference.clip(aoi), {'min': -10, 'max': 10, 'palette': cc.cwr}, 'Difference')
m

Map(center=[9.973546567083341, -84.19370076516422], controls=(WidgetControl(options=['position', 'transparent_…